# Time Series Dictionary Sequence
This Notebook is creating a Dictionary for the different concepts and is adding the new "terms" to the data frame. Additionaly are the sequences per patient and per patient per day created.

In [ ]:
import os
import pyarrow.parquet as pq
import numpy as np
import pandas as pd
import pickle
from sklearn.utils import shuffle
import os
import csv
#import spacy
import multiprocessing
import time
from gensim.models import Word2Vec
from gensim.models import FastText
import pandas as pd
import numpy as np
import pyarrow.parquet as pq

In [ ]:
#load timeseries data and shuffle it, so that the order of events on the same day is random
timeseries_data = pq.read_table('Cohort/Time_Series/all_time_series_woProcedures.parquet').to_pandas() # raw dataframe
#timeseries_data=timeseries_data.head(3000)
timeseries_data=shuffle(timeseries_data)

In [ ]:
timeseries_data

In [ ]:
#sorting the dataframe again by mrn and time_delta_in_days
timeseries_data=timeseries_data.sort_values(by=['medical_record_number', 'time_delta_in_days'])

In [ ]:
timeseries_data

In [ ]:
#select the timeframe that should be considered for the time series 
timeseries_data = timeseries_data.drop(timeseries_data[(timeseries_data['time_delta_in_days'] < (-730))].index)

In [ ]:
timeseries_data

In [ ]:
#get unique events/concepts and create empty dictionary
unique_values = timeseries_data.event.unique()
unique_concepts_dictionary ={}
len(unique_values)

In [ ]:
for i,val in enumerate(unique_values):
    unique_concepts_dictionary[val]= i

In [ ]:
unique_concepts_dictionary

In [ ]:
#generate a df dictionary 
df = pd.DataFrame(list(unique_concepts_dictionary.items()),columns = ['Term','term_id'])
df

In [ ]:
#save the dictionary 
df.to_parquet('Cohort/Time_Series/all_time_series_woProcedures_dictionary.parquet')

In [ ]:
#create a column for the Term ID 
timeseries_data['unique_concept']=''

In [ ]:
#add the term to the events 
for key in unique_concepts_dictionary:
    print('Current key:',key)
    print('*'*100)
    timeseries_data['unique_concept'] = np.where(timeseries_data['event'] == key, unique_concepts_dictionary[key] , timeseries_data['unique_concept'])
    print(timeseries_data[timeseries_data['event']==key].head())

In [ ]:
timeseries_data

In [ ]:
#save the dataframe
timeseries_data.to_parquet('Cohort/Time_Series/all_time_series_woProcedures_HFonset_wVocabular.parquet')
#timeseries_data=timeseries_data.head(10)
timeseries_data

In [ ]:
#read the dataframe
timeseries_data = pq.read_table('Cohort/Time_Series/all_time_series_woProcedures_HFonset_wVocabular.parquet').to_pandas()
#timeseries_data=timeseries_data.head(100)
timeseries_data

# Remove MRN which do not have any Lab Values

In [ ]:
#mrn without labvalues: 
mrn_without_lab=pq.read_table('Cohort/Feature_Extraction/Unsupervised_ALL_HF/LabValue_after_onset_HF_ALL_mmm_0_8_missing_values').to_pandas()

In [ ]:
mrn_list=mrn_without_lab['medical_record_number'].to_list()
len(mrn_list)

In [ ]:
timeseries_data = timeseries_data[~timeseries_data['medical_record_number'].isin(mrn_list)]
timeseries_data

# Timeseries per Patient 

In [ ]:
#buid dataframe timeseries data per patient 
timeseries_data_per_patient = timeseries_data[['medical_record_number','unique_concept']].groupby(['medical_record_number']).aggregate(lambda x: list(x)).reset_index()
timeseries_data_per_patient

In [ ]:
timeseries_data_per_patient.to_parquet('Cohort/Time_Series/all_time_series_woProcedures_timeseries_data_per_patient.parquet')

In [ ]:
# plot sequence length of the patient: 
seqlength=[]
for index, row in timeseries_data_per_patient.iterrows(): 
    seqlength.append(len(row['unique_concept']))
seqlength

In [ ]:
import matplotlib.pyplot as plt
import statistics 

plt.hist(seqlength, bins =1000)
plt.show()

In [ ]:
statistics.mean(seqlength)

In [ ]:
#number of patient which have less than x events
sum(i < 100 for i in seqlength)

# Time Series per Patient per Day 

In [ ]:
timeseries_data_per_patient_per_day = timeseries_data[['medical_record_number','time_delta_in_days','unique_concept']].groupby(['medical_record_number','time_delta_in_days']).aggregate(lambda x: list(x)).reset_index()
timeseries_data_per_patient_per_day

In [ ]:
timeseries_data_per_patient_per_day.to_parquet('Cohort/Time_Series/all_time_series_woProcedures_timeseries_data_per_patient_per_day.parquet')

In [ ]:
days_per_patient=timeseries_data_per_patient_per_day['medical_record_number'].value_counts()

In [ ]:
days_per_patient

In [ ]:
plt.hist(days_per_patient, bins =1000)
plt.show()

In [ ]:
statistics.median(days_per_patient)

In [ ]:
#number of patient which have less than x days
sum(i < 11 for i in days_per_patient)